# Diet Recommendation System

Diet Recommendation for healthy people using Fuzzy Logic combined with K Nearest Neighbor.
Based on https://www.mecs-press.org/ijitcs/ijitcs-v13-n4/IJITCS-V13-N4-1.pdf

In [63]:

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [3]:
# Download first foodcom dataset from link in sources.md inside Docs folder
df_recipes = pd.read_csv('Data/foodcom/recipes.csv')

In [4]:
df_recipes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522517 entries, 0 to 522516
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   RecipeId                    522517 non-null  int64  
 1   Name                        522517 non-null  object 
 2   AuthorId                    522517 non-null  int64  
 3   AuthorName                  522517 non-null  object 
 4   CookTime                    439972 non-null  object 
 5   PrepTime                    522517 non-null  object 
 6   TotalTime                   522517 non-null  object 
 7   DatePublished               522517 non-null  object 
 8   Description                 522512 non-null  object 
 9   Images                      522516 non-null  object 
 10  RecipeCategory              521766 non-null  object 
 11  Keywords                    505280 non-null  object 
 12  RecipeIngredientQuantities  522514 non-null  object 
 13  RecipeIngredie

In [5]:
dataset=df_recipes.copy()
columns=['RecipeId','Name','CookTime','PrepTime','TotalTime','RecipeIngredientParts',
         'Calories','FatContent','SaturatedFatContent','CholesterolContent',
         'SodiumContent','CarbohydrateContent','FiberContent','SugarContent',
         'ProteinContent','RecipeInstructions']
dataset=dataset[columns]

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522517 entries, 0 to 522516
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   RecipeId               522517 non-null  int64  
 1   Name                   522517 non-null  object 
 2   CookTime               439972 non-null  object 
 3   PrepTime               522517 non-null  object 
 4   TotalTime              522517 non-null  object 
 5   RecipeIngredientParts  522517 non-null  object 
 6   Calories               522517 non-null  float64
 7   FatContent             522517 non-null  float64
 8   SaturatedFatContent    522517 non-null  float64
 9   CholesterolContent     522517 non-null  float64
 10  SodiumContent          522517 non-null  float64
 11  CarbohydrateContent    522517 non-null  float64
 12  FiberContent           522517 non-null  float64
 13  SugarContent           522517 non-null  float64
 14  ProteinContent         522517 non-nu

In [6]:
max_Calories=2000
max_daily_fat=100
max_daily_Saturatedfat=13
max_daily_Cholesterol=300
max_daily_Sodium=2300
max_daily_Carbohydrate=325
max_daily_Fiber=40
max_daily_Sugar=40
max_daily_Protein=200
max_list=[max_Calories, max_daily_fat, 
          max_daily_Saturatedfat, max_daily_Cholesterol, 
          max_daily_Sodium, max_daily_Carbohydrate, 
          max_daily_Fiber, max_daily_Sugar, max_daily_Protein]

In [7]:
extracted_data = dataset.copy()
for column, maximum in zip(extracted_data.columns[6:15], max_list):
    extracted_data = extracted_data[extracted_data[column]<maximum]

In [8]:
extracted_data.shape

(375703, 16)

## Fuzzy Logic
Get the required amount of macronutrients according to Age, BMI, and physical activity level through Fuzzy Logic


In [9]:
# !pip install -U scikit-fuzzy matplotlib
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [71]:
# Define Fuzzy sets for input variables
bmi = ctrl.Antecedent(np.arange(0, 41, 1), 'bmi')
age = ctrl.Antecedent(np.arange(0, 101, 1), 'age')
pal = ctrl.Antecedent(np.arange(0, 5, 1), 'pal')

# Define Fuzzy sets for output variables
calories = ctrl.Consequent(np.arange(0, 601, 1), 'calories')
fat = ctrl.Consequent(np.arange(0, 41, 1), 'fat')
fat_saturated = ctrl.Consequent(np.arange(0, 31, 1), 'fat_saturated')
cholesterol = ctrl.Consequent(np.arange(0, 101, 1), 'cholesterol')
sodium = ctrl.Consequent(np.arange(0, 501, 1), 'sodium')
carbohydrate = ctrl.Consequent(np.arange(0, 51, 1), 'carbohydrate')
fiber = ctrl.Consequent(np.arange(0, 31, 1), 'fiber')
sugar = ctrl.Consequent(np.arange(0, 51, 1), 'sugar')
protein = ctrl.Consequent(np.arange(0, 41, 1), 'protein')

# Define fuzzy membership functions for inputs
bmi['underweight'] = fuzz.trimf(bmi.universe, [0, 0, 18.5])
bmi['normal'] = fuzz.trimf(bmi.universe, [18.5, 21.5, 24.9])
bmi['overweight'] = fuzz.trimf(bmi.universe, [25.0, 27.5, 29.9])
bmi['obesity'] = fuzz.trimf(bmi.universe, [30.0, 35.0, 40.0])

age['young'] = fuzz.trimf(age.universe, [0, 0, 21])
age['adult'] = fuzz.trimf(age.universe, [18, 35, 55])
age['old'] = fuzz.trimf(age.universe, [50, 75, 100])

pal['low'] = fuzz.trimf(pal.universe, [0, 0, 2])
pal['medium'] = fuzz.trimf(pal.universe, [1, 2, 3])
pal['high'] = fuzz.trimf(pal.universe, [2, 4, 4])

# Define fuzzy membership functions for outputs
calories['low'] = fuzz.trimf(calories.universe, [0, 150, 350])
calories['medium'] = fuzz.trimf(calories.universe, [300, 450, 525])
calories['high'] = fuzz.trimf(calories.universe, [450, 525, 600])

fat['low'] = fuzz.trimf(fat.universe, [0, 10, 20])
fat['medium'] = fuzz.trimf(fat.universe, [15, 20, 30])
fat['high'] = fuzz.trimf(fat.universe, [25, 30, 40])

fat_saturated['low'] = fuzz.trimf(fat_saturated.universe, [0, 10, 15])
fat_saturated['medium'] = fuzz.trimf(fat_saturated.universe, [10, 15, 20])
fat_saturated['high'] = fuzz.trimf(fat_saturated.universe, [15, 25, 30])

cholesterol['low'] = fuzz.trimf(cholesterol.universe, [0, 25, 50])
cholesterol['medium'] = fuzz.trimf(cholesterol.universe, [30, 50, 75])
cholesterol['high'] = fuzz.trimf(cholesterol.universe, [50, 75, 100])

sodium['low'] = fuzz.trimf(sodium.universe, [0, 100, 250])
sodium['medium'] = fuzz.trimf(sodium.universe, [150, 250, 400])
sodium['high'] = fuzz.trimf(sodium.universe, [300, 400, 500])

carbohydrate['low'] = fuzz.trimf(carbohydrate.universe, [0, 10, 20])
carbohydrate['medium'] = fuzz.trimf(carbohydrate.universe, [15, 25, 35])
carbohydrate['high'] = fuzz.trimf(carbohydrate.universe, [30, 40, 50])

fiber['low'] = fuzz.trimf(fiber.universe, [0, 5, 15])
fiber['medium'] = fuzz.trimf(fiber.universe, [10, 15, 25])
fiber['high'] = fuzz.trimf(fiber.universe, [20, 25, 30])

sugar['low'] = fuzz.trimf(sugar.universe, [0, 10, 20])
sugar['medium'] = fuzz.trimf(sugar.universe, [15, 25, 35])
sugar['high'] = fuzz.trimf(sugar.universe, [30, 40, 50])

protein['low'] = fuzz.trimf(protein.universe, [0, 10, 20])
protein['medium'] = fuzz.trimf(protein.universe, [15, 20, 30])
protein['high'] = fuzz.trimf(protein.universe, [25, 30, 40])

# Define fuzzy rules
rule1 = ctrl.Rule(age['young'] & bmi['underweight'] & pal['low'], 
                  (calories['high'], fat['medium'], fat_saturated['medium'], 
                   cholesterol['medium'], sodium['medium'], carbohydrate['high'], 
                   fiber['high'], sugar['low'], protein['high']))
rule2 = ctrl.Rule(age['young'] & bmi['normal'] & pal['low'], 
                  (calories['medium'], fat['medium'], fat_saturated['low'], 
                   cholesterol['low'], sodium['low'], carbohydrate['medium'], 
                   fiber['high'], sugar['medium'], protein['medium']))
rule3 = ctrl.Rule(age['young'] & bmi['overweight'] & pal['low'], 
                  (calories['low'], fat['low'], fat_saturated['low'], 
                   cholesterol['low'], sodium['low'], carbohydrate['low'], 
                   fiber['high'], sugar['low'], protein['low']))
rule3 = ctrl.Rule(age['young'] & bmi['obesity'] & pal['low'], 
                  (calories['low'], fat['low'], fat_saturated['low'], 
                   cholesterol['low'], sodium['low'], carbohydrate['low'], 
                   fiber['high'], sugar['low'], protein['low']))

rule4 = ctrl.Rule(age['adult'] & bmi['underweight'] & pal['low'], 
                  (calories['high'], fat['high'], fat_saturated['medium'], 
                   cholesterol['medium'], sodium['medium'], carbohydrate['high'], 
                   fiber['high'], sugar['low'], protein['high']))
rule5 = ctrl.Rule(age['adult'] & bmi['normal'] & pal['low'], 
                  (calories['medium'], fat['medium'], fat_saturated['low'], 
                   cholesterol['low'], sodium['low'], carbohydrate['medium'], 
                   fiber['medium'], sugar['medium'], protein['medium']))
rule6 = ctrl.Rule(age['adult'] & bmi['overweight'] & pal['low'], 
                  (calories['low'], fat['low'], fat_saturated['low'], 
                   cholesterol['low'], sodium['low'], carbohydrate['low'], 
                   fiber['high'], sugar['low'], protein['low']))
rule7 = ctrl.Rule(age['adult'] & bmi['obesity'] & pal['low'], 
                  (calories['low'], fat['low'], fat_saturated['low'], 
                   cholesterol['low'], sodium['low'], carbohydrate['low'], 
                   fiber['high'], sugar['low'], protein['low']))

rule8 = ctrl.Rule(age['old'] & bmi['underweight'] & pal['low'], 
                  (calories['high'], fat['high'], fat_saturated['medium'], 
                   cholesterol['medium'], sodium['medium'], carbohydrate['high'], 
                   fiber['high'], sugar['low'], protein['high']))
rule9 = ctrl.Rule(age['old'] & bmi['normal'] & pal['low'], 
                  (calories['medium'], fat['medium'], fat_saturated['low'], 
                   cholesterol['low'], sodium['low'], carbohydrate['medium'], 
                   fiber['medium'], sugar['medium'], protein['medium']))
rule10 = ctrl.Rule(age['old'] & bmi['overweight'] & pal['low'], 
                   (calories['low'], fat['low'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['low'], 
                    fiber['high'], sugar['low'], protein['low']))
rule11 = ctrl.Rule(age['old'] & bmi['obesity'] & pal['low'], 
                   (calories['low'], fat['low'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['low'], 
                    fiber['high'], sugar['low'], protein['low']))


rule12 = ctrl.Rule(age['young'] & bmi['underweight'] & pal['medium'], 
                   (calories['high'], fat['high'], fat_saturated['medium'], 
                    cholesterol['medium'], sodium['medium'], carbohydrate['high'], 
                    fiber['high'], sugar['low'], protein['high']))
rule13 = ctrl.Rule(age['young'] & bmi['normal'] & pal['medium'], 
                   (calories['medium'], fat['medium'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['medium'], 
                    fiber['medium'], sugar['medium'], protein['medium']))
rule14 = ctrl.Rule(age['young'] & bmi['overweight'] & pal['medium'], 
                   (calories['low'], fat['low'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['low'], 
                    fiber['high'], sugar['low'], protein['low']))
rule15 = ctrl.Rule(age['young'] & bmi['obesity'] & pal['medium'], 
                   (calories['low'], fat['low'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['low'], 
                    fiber['high'], sugar['low'], protein['low']))

rule16 = ctrl.Rule(age['adult'] & bmi['underweight'] & pal['medium'], 
                   (calories['high'], fat['high'], fat_saturated['medium'], 
                    cholesterol['medium'], sodium['medium'], carbohydrate['high'], 
                    fiber['high'], sugar['low'], protein['high']))
rule17 = ctrl.Rule(age['adult'] & bmi['normal'] & pal['medium'], 
                   (calories['medium'], fat['medium'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['medium'], 
                    fiber['medium'], sugar['medium'], protein['medium']))
rule18 = ctrl.Rule(age['adult'] & bmi['overweight'] & pal['medium'], 
                   (calories['low'], fat['low'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['low'], 
                    fiber['high'], sugar['low'], protein['low']))
rule19 = ctrl.Rule(age['adult'] & bmi['obesity'] & pal['medium'], 
                   (calories['low'], fat['low'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['low'], 
                    fiber['high'], sugar['low'], protein['low']))

rule20 = ctrl.Rule(age['old'] & bmi['underweight'] & pal['medium'], 
                   (calories['high'], fat['high'], fat_saturated['medium'], 
                    cholesterol['medium'], sodium['medium'], carbohydrate['high'], 
                    fiber['high'], sugar['low'], protein['high']))
rule21 = ctrl.Rule(age['old'] & bmi['normal'] & pal['medium'], 
                   (calories['medium'], fat['medium'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['medium'], 
                    fiber['medium'], sugar['medium'], protein['medium']))
rule22 = ctrl.Rule(age['old'] & bmi['overweight'] & pal['medium'], 
                   (calories['low'], fat['low'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['low'], 
                    fiber['high'], sugar['low'], protein['low']))
rule23 = ctrl.Rule(age['old'] & bmi['obesity'] & pal['medium'], 
                   (calories['low'], fat['low'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['low'], 
                    fiber['high'], sugar['low'], protein['low']))


rule24 = ctrl.Rule(age['young'] & bmi['underweight'] & pal['high'], 
                   (calories['high'], fat['high'], fat_saturated['medium'], 
                    cholesterol['medium'], sodium['medium'], carbohydrate['high'], 
                    fiber['high'], sugar['low'], protein['high']))
rule25 = ctrl.Rule(age['young'] & bmi['normal'] & pal['high'], 
                   (calories['medium'], fat['medium'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['medium'], 
                    fiber['medium'], sugar['medium'], protein['medium']))
rule26 = ctrl.Rule(age['young'] & bmi['overweight'] & pal['high'], 
                   (calories['low'], fat['low'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['low'], 
                    fiber['high'], sugar['low'], protein['low']))
rule27 = ctrl.Rule(age['young'] & bmi['obesity'] & pal['high'], 
                   (calories['low'], fat['low'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['low'], 
                    fiber['high'], sugar['low'], protein['low']))

rule28 = ctrl.Rule(age['adult'] & bmi['underweight'] & pal['high'], 
                   (calories['high'], fat['high'], fat_saturated['medium'], 
                    cholesterol['medium'], sodium['medium'], carbohydrate['high'], 
                    fiber['high'], sugar['low'], protein['high']))
rule29 = ctrl.Rule(age['adult'] & bmi['normal'] & pal['high'], 
                   (calories['medium'], fat['medium'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['medium'], 
                    fiber['medium'], sugar['medium'], protein['medium']))
rule30 = ctrl.Rule(age['adult'] & bmi['overweight'] & pal['high'], 
                   (calories['low'], fat['low'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['low'], 
                    fiber['high'], sugar['low'], protein['low']))
rule31 = ctrl.Rule(age['adult'] & bmi['obesity'] & pal['high'], 
                   (calories['low'], fat['low'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['low'], 
                    fiber['high'], sugar['low'], protein['low']))

rule32 = ctrl.Rule(age['old'] & bmi['underweight'] & pal['high'], 
                   (calories['high'], fat['high'], fat_saturated['medium'], 
                    cholesterol['medium'], sodium['medium'], carbohydrate['high'], 
                    fiber['high'], sugar['low'], protein['high']))
rule33 = ctrl.Rule(age['old'] & bmi['normal'] & pal['high'], 
                   (calories['medium'], fat['medium'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['medium'], 
                    fiber['medium'], sugar['medium'], protein['medium']))
rule34 = ctrl.Rule(age['old'] & bmi['overweight'] & pal['high'], 
                   (calories['low'], fat['low'], fat_saturated['low'],
                    cholesterol['low'], sodium['low'], carbohydrate['low'], 
                    fiber['high'], sugar['low'], protein['low']))
rule35 = ctrl.Rule(age['old'] & bmi['obesity'] & pal['high'], 
                   (calories['low'], fat['low'], fat_saturated['low'], 
                    cholesterol['low'], sodium['low'], carbohydrate['low'], 
                    fiber['high'], sugar['low'], protein['low']))

# Implement the fuzzy inference system
diet_ctrl = ctrl.ControlSystem(
    [rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10, 
     rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18, rule19, 
     rule20, rule21, rule22, rule23, rule24, rule25, rule26, rule27, rule28,
     rule29, rule30, rule31, rule32, rule33, rule34, rule35])
diet_simulation = ctrl.ControlSystemSimulation(diet_ctrl)

In [81]:
# Test fuzzy logic system with sample inputs
# Example input
diet_simulation.input['bmi'] = 21.9
diet_simulation.input['age'] = 23
diet_simulation.input['pal'] = 0

# Compute the output
diet_simulation.compute()

# Print the output values
print(f"Calories:       {round(diet_simulation.output['calories'], 2)}")
print(f"Fat:            {round(diet_simulation.output['fat'], 2)}")
print(f"Fat Saturated:  {round(diet_simulation.output['fat_saturated'], 2)}")
print(f"Cholesterol:    {round(diet_simulation.output['cholesterol'], 2)}")
print(f"Sodium:         {round(diet_simulation.output['sodium'], 2)}")
print(f"Carbohydrate:   {round(diet_simulation.output['carbohydrate'], 2)}")
print(f"Fiber:          {round(diet_simulation.output['fiber'], 2)}")
print(f"Sugar:          {round(diet_simulation.output['sugar'], 2)}")
print(f"Protein:        {round(diet_simulation.output['protein'], 2)}")

Calories:       417.7
Fat:            22.15
Fat Saturated:  7.85
Cholesterol:    25.0
Sodium:         121.53
Carbohydrate:   25.0
Fiber:          17.15
Sugar:          25.0
Protein:        22.15


In [73]:
diet_simulation.output

OrderedDict([('calories', 417.69776876267645),
             ('fat', 22.15348208248817),
             ('fat_saturated', 7.846517917511833),
             ('cholesterol', 24.999999999999986),
             ('sodium', 121.53482082488134),
             ('carbohydrate', 25.000000000000007),
             ('fiber', 17.15348208248817),
             ('sugar', 25.000000000000007),
             ('protein', 22.15348208248817)])

## KNN
Recommend meals using the output of the Fuzzy Logic System as input for KNN

In [74]:
# Get numerical columns and scale
scaler = StandardScaler()
prep_data = scaler.fit_transform(extracted_data.iloc[:,6:15].to_numpy())
prep_data[0]

array([-0.55093359, -0.91281917, -0.77924852, -0.68354951, -0.89654601,
        0.37502226,  0.15672078,  2.35502102, -0.68338127])

In [75]:
np.array(list(diet_simulation.output.values())).reshape(1,-1)

array([[417.69776876,  22.15348208,   7.84651792,  25.        ,
        121.53482082,  25.        ,  17.15348208,  25.        ,
         22.15348208]])

In [76]:
# Create model
model = NearestNeighbors(metric='minkowski', p=2, algorithm='auto')
model.fit(prep_data)

# Building pipeline
parameters = {
    'n_neighbors':5,
    'return_distance':False
}
transformer = FunctionTransformer(model.kneighbors, kw_args=parameters)
pipeline = Pipeline([('std_scaler', scaler), ('NN', transformer)])

# Apply pipeline to _input
_input=np.array(list(diet_simulation.output.values())).reshape(1,-1)
recommendations = extracted_data.iloc[pipeline.transform(_input)[0]]

In [77]:
recommendations

,RecipeId,Name,CookTime,PrepTime,TotalTime,RecipeIngredientParts,Calories,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeInstructions
124050,130421,Knife and Fork Grilled Vegetable Salad,PT25M,PT10M,PT35M,"c(""garlic clove"", ""kosher salt"", ""extra virgin...",403.0,25.8,7.9,22.4,181.6,36.5,16.1,14.8,14.9,"c(""Light a grill."", ""In a mortar, pound the ha..."
388548,402536,Jalapeno Cream Cheese Appetizer,PT5M,PT120H,PT120H5M,"c(""cream cheese"", ""sugar"")",334.5,16.7,8.7,41.6,117.6,42.9,15.9,28.1,10.5,"c(""Drain jalapenos of ALL juice. There should ..."
443151,459497,Romaine and Chicken Muscle Salad,NaN,PT10M,PT10M,"c(""romaine lettuce"", ""red apple"", ""celery rib""...",439.1,23.8,4.6,63.8,197.9,36.7,17.0,18.0,27.2,"c(""Toss together lettuce, apple, and celery; t..."
416981,432295,Roast Eggplant Salad With Goat Cheese,PT20M,PT15M,PT35M,"c(""eggplants"", ""extra virgin olive oil"", ""cher...",443.8,30.1,8.9,22.4,174.0,37.8,17.9,15.9,13.4,"c(""Heat the oven to 400°F Brush the eggplant s..."
350690,363627,Colombian Recipes - Cabbage Tamales,PT45M,PT15M,PT1H,"c(""cabbage"", ""ground pork"", ""green peas"", ""car...",401.5,16.6,6.0,71.1,256.9,37.9,14.2,21.6,30.1,"c(""Prepare a stew of onions and tomatoes, chop..."
